In [1]:
# 다음 뉴스 or MBC 뉴스
def whichNews(driver):
    try:
        a1 = driver.find_element_by_css_selector('#content > div > section.wrap_article > article > div.wrap_title')
#         print('MBC')
        return 'mbc'
    except:
        a1 = driver.find_element_by_css_selector('#cSub > div > h3')
        # print('다음')
        return 'daum'

In [2]:
# MBC - 더보기 계속 클릭하기
def clickMoreMBC(driver):
    import time
    
    while True:
        try:
            more_button = driver.find_element_by_css_selector('#commentBox > div.list_comment > div > button')
            more_button.click()
            # print('클릭 성공')
            time.sleep(2)
        except:
            break

In [3]:
# 다음 - 더보기 계속 클릭하기
def clickMoreDaum(driver):
    import time
    from selenium.webdriver.common.keys import Keys

    while True:
        try:
            more_button = driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > button')
            # more_button.sendKeys(Keys.ENTER)
            more_button.click()
            print('클릭 성공')
            time.sleep(2)
        except:
            break

In [4]:
# 다음 하나의 기사에 댓글이 있는지 확인
def areAnyCommentsDaum(driver):
    a1 = driver.find_element_by_css_selector('#alexCounter > span').text
    #print(a1)
    
    if a1 == '0':
        # print('댓글 없음')
        return False
    else:
        # print('댓글 있음')
        return True

In [5]:
# MBC - 하나의 기사에 댓글이 있는지 확인
def areAnyCommentsMBC(driver):
    a1 = driver.find_element_by_css_selector('#commentBox > div.comment_numb').text
    #print(a1)
    
    if a1 == '0':
        # print('댓글 없음')
        return False
    else:
        # print('댓글 있음')
        return True

In [6]:
# 다음 - 한 기사의 댓글 추출
def getCommentDaum(driver):
    
    import os
    import time
    import pandas as pd
    from selenium.webdriver.common.keys import Keys

    
    # 데이터를 담을 딕셔너리
    comment_dict = {
        '제목' : [],
        '날짜' : [],
        '작성자' : [],
        '작성일' : [],
        '댓글' : []
    }
    
    # 기사 제목 & 날짜 태그
    a1 = driver.find_element_by_css_selector('#cSub > div ')
    
    # 기사 제목 
    data1 = a1.find_element_by_css_selector('h3').text
    # print(data1)

    # 날짜 
    data2 = a1.find_element_by_css_selector('span > span:nth-child(2) > span').text
    # print(data2)
    
    # 댓글 태그
    a2 = driver.find_elements_by_css_selector('#alex-area > div > div > div > div.cmt_box > ul.list_comment > li')
    # print(len(a2))
        
    # 댓글 수만큼 반복
    for a3 in a2:

        # 작성자
        data3 = a3.find_element_by_css_selector('div > strong > span > a').text
        # print(data3)

        # 댓글 작성일
        data4 = a3.find_element_by_css_selector('div > strong > span > span.txt_date').text
        # print(data4)
        
        # 댓글 
        data5 = a3.find_element_by_css_selector('div > p').text
        # print(data5)
        
        # 저장
        comment_dict['제목'].append(data1)
        comment_dict['날짜'].append(data2)
        comment_dict['작성자'].append(data3)
        comment_dict['작성일'].append(data4)
        comment_dict['댓글'].append(data5)

        # 답글 버튼 
        a4 = a3.find_element_by_css_selector('div > div > span.box_inner > button').text
        print(a4[-2:])
        
        # 답글 버튼 클릭하기
        if a4[-2:] != '작성':
            print('클릭시도')
            reply_button = a3.find_element_by_css_selector('div > div.box_reply > span.box_inner > button')        
            reply_button.send_keys(Keys.ENTER)
#             reply_button.click()
            time.sleep(2)
            print('클릭 완료')

            # 답글 태그
            a5 = a3.find_elements_by_css_selector('div > div.reply_wrap > div.cont_wrap > ul.list_reply > li')
            # print(len(a5))
            
            # 답글 수만큼 반복
            for a6 in a5:

                # 작성자
                data3 = a6.find_element_by_css_selector('div > div > strong > span.info_author > a').text
                # print(data3)
                
                # 작성일
                data4 = a6.find_element_by_css_selector('div > div > strong > span.info_author > span.txt_date').text
                # print(data4)
                
                # 답글
                data5 = a6.find_element_by_css_selector('div > div > p').text
                # print(data5)

                comment_dict['제목'].append(data1)
                comment_dict['날짜'].append(data2)
                comment_dict['작성자'].append(data3)
                comment_dict['작성일'].append(data4)
                comment_dict['댓글'].append(data5)
        
    df = pd.DataFrame(comment_dict)
    df['날짜'] = df['날짜'].str.replace('입력 ', '')
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['작성일'] = pd.to_datetime(df['작성일'])
    display(df)

    FILENAME = 'mbc_comment_daum.csv'
    if os.path.exists(FILENAME) == False:
        # 파일이 없을 경우
        df.to_csv(FILENAME, encoding='utf-8-sig', index=False)
    else:
        # mode='a' : 기존 것 뒤에다 붙여줌
        df.to_csv(FILENAME, encoding='utf-8-sig', index=False, header=False, mode='a')

    return True


In [7]:
# MBC - 한 기사의 댓글 추출
def getCommentMBC(driver):
    
    import os
    import time
    import pandas as pd
    
    # 데이터를 담을 딕셔너리
    comment_dict = {
        '제목' : [],
        '날짜' : [],
        '작성자' : [],
        '작성일' : [],
        '댓글' : []
    }
    
    # 기사 제목 & 날짜 태그
    a1 = driver.find_element_by_css_selector('#content > div > section.wrap_article > article > div.wrap_title')
    
    # 기사 제목 
    data1 = a1.find_element_by_css_selector('h2').text
    # print(data1)
    
    # 날짜 
    data2 = a1.find_element_by_css_selector('div.ui_article > div.date > span:nth-child(1)').text
    # print(data2)
    
    # 댓글 태그
    a2 = driver.find_elements_by_css_selector('#commentBox > div.list_comment > ul > li')
    # print(len(a2))
        
    # 댓글 수만큼 반복
    for a3 in a2:
        
        # 작성자
        data3 = a3.find_element_by_css_selector('div.u_comment_box > div.user > span.user_id').text
        # print(data3)
        
        # 댓글 작성일
        data4 = a3.find_element_by_css_selector('div.u_comment_box > div.user > span.user_date').text
        # print(data4)
        
        # 댓글 
        data5 = a3.find_element_by_css_selector('div.u_comment_box > div.user_content > div').text
        # print(data5)
        
        # 저장
        comment_dict['제목'].append(data1)
        comment_dict['날짜'].append(data2)
        comment_dict['작성자'].append(data3)
        comment_dict['작성일'].append(data4)
        comment_dict['댓글'].append(data5)
        
        # 답글 버튼 
        a4 = a3.find_element_by_css_selector('div.u_comment_box > div.user_ui_btn > button').text
        # print(a4.split()[-1])
        
        # 답글 버튼 클릭하기
        if a4.split()[-1] != '작성':
            reply_button = a3.find_element_by_css_selector('div.u_comment_box > div.user_ui_btn > button')        
            driver.execute_script("arguments[0].click();", reply_button)  #자바 명령어 실행
            time.sleep(1)
            # print('클릭 완료')
            
            # 답글 태그
            a5 = a3.find_elements_by_css_selector('div.u_wrap_reply > div')
            # print(len(a5))
            
            # 답글 수만큼 반복
            for a6 in a5[:-1]:
                
                # 작성자
                data3 = a6.find_element_by_css_selector('div.user > span.user_id').text
                # print(data3)
                
                # 작성일
                data4 = a6.find_element_by_css_selector('div.user > span.user_date').text
                # print(data4)
                
                # 답글
                data5 = a6.find_element_by_css_selector('div.user_content > div').text
                # print(data5)

                comment_dict['제목'].append(data1)
                comment_dict['날짜'].append(data2)
                comment_dict['작성자'].append(data3)
                comment_dict['작성일'].append(data4)
                comment_dict['댓글'].append(data5)
        
    df = pd.DataFrame(comment_dict)
    df['날짜'] = df['날짜'].str.replace('입력 ', '')
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['작성일'] = pd.to_datetime(df['작성일'])
    display(df)

    FILENAME = 'mbc_comment.csv'
    if os.path.exists(FILENAME) == False:
        # 파일이 없을 경우
        df.to_csv(FILENAME, encoding='utf-8-sig', index=False)
    else:
        # mode='a' : 기존 것 뒤에다 붙여줌
        df.to_csv(FILENAME, encoding='utf-8-sig', index=False, header=False, mode='a')

    return True


In [ ]:
import pandas as pd
from selenium import webdriver as wd
from IPython.display import clear_output
from Code_뉴스댓글수집.DaumNews import getDaumNewsUrlDF

# 후보자명
CANDIDATES = ['이재명', '윤석열', '이낙연', '홍준표', '안철수', '정세균', '유승민', '심상정', '추미애', '황교안', '김부겸']
COLOPHON_STR = {'조선일보' : 'chosun', '중앙일보' : 'joongang', '동아일보' : 'donga',
                'JTBC' : 'jtbc', 'MBC' : 'mbc',}

for i in range(len(CANDIDATES)):
    
    # 검색어
    KEYWORD = f'대선 {CANDIDATES[i]}'
    print(f'수집한 기사의 키워드는 : {KEYWORD}')

    # 언론
    # 조선일보, 중앙일보, 동아일보, JTBC, 경향신문, MBC 택1
    COLOPHON = 'MBC'

    # 날짜 (YYYYMMDhhmmss)
    dayStart = '20210301000000'
    dayEnd   = '20210514000000'

    # 기사 리스트 가져오기
    df = getDaumNewsUrlDF(KEYWORD=KEYWORD, COLOPHON=COLOPHON, dayStart=dayStart, dayEnd=dayEnd, page=1)
    
    # 크롬 옵션 생성
    options = wd.ChromeOptions()
    # 창 숨기는 옵션 추가
    options.add_argument("headless")
    
    num_link = df.shape[0]

    # 웹 드라이버
    driver = wd.Chrome('./chromedriver.exe', options=options)
    idx = 0

    while True :

        driver.implicitly_wait(20)
        clear_output(wait=True)

        driver.get(df.loc[idx].values[0])
#         driver.get('https://news.v.daum.net/v/20210419193026094?f=o')
        driver.refresh()
        source = whichNews(driver)
        if source == 'mbc':
            # 댓글이 없으면
            if not areAnyCommentsMBC(driver):
                print(f'{idx}번째 기사 댓글 없음 다음으로~')

            # 댓글이 있으면
            else:
                print(f'{idx}번째 기사 댓글 수집 중')

                # 더보기 버튼 클릭하기
                clickMoreMBC(driver)
                # 댓글 수집하기
                getCommentMBC(driver)
        else:
            # 댓글이 없으면
            if not areAnyCommentsDaum(driver):
                print(f'{idx}번째 기사 댓글 없음 다음으로~')
                
            else:
                print(f'{idx}번째 기사 댓글 수집 중')
                
                # 더보기 버튼 클릭하기
                clickMoreDaum(driver)
                
                # 댓글 수집하기
                getCommentDaum(driver)
        
        idx += 1
        if idx == num_link:
            driver.quit()
            
            
    # 파일명 변경
    import os
    os.rename(f'{COLOPHON}_link.csv', f'{COLOPHON_STR[COLOPHON]}_link_{CANDIDATES[i]}.csv')
    os.rename(f'{COLOPHON_STR[COLOPHON]}_comment.csv', f'{COLOPHON_STR[COLOPHON]}_comment_{CANDIDATES[i]}.csv')
    os.rename(f'{COLOPHON_STR[COLOPHON]}_comment_daum.csv', f'{COLOPHON_STR[COLOPHON]}_comment__daum_{CANDIDATES[i]}.csv')
    print(f'{KEYWORD} 키워드 댓글 저장 완료')
    break


7번째 기사 댓글 수집 중


In [ ]:
print('MBC_link.csv')